# Social Network Analysis - Project sna_topic_11

In [ ]:
import numpy as np
np.random.seed(42)
import random
random.seed(42)

import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from itertools import compress
import pickle
from itertools import compress, product
import datetime

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
pd.reset_option("^display")

In [ ]:
pd.set_option('display.float_format', '{:20,.4f}'.format)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 2000)

## The Data

The data set is provided by Der Standard, one of the top Austrian newspapers.
In the online Standard people can post comments below articles and up/down vote comments.

1. There are two files containing all **postings** to articles in May 2019 (due to high amount of data, the postings were split into two files). The respective file also contains additionally meta-data of the postings and articles and some details about the users who composed the postings.

2. There are two files containing all **votes** for the postings in point 1 (due to high amount of data, the votes were split into two files). The respective file also contains information whether the vote was negative or positive and some details about the user who did the voting.

3. There is one file containing **following and ignoring relationships** among all the users who posted (see point 1) or voted (see point 2) to articles published in May 2019. A following relationship (i.e., the user with the `ID_CommunityIdentity` given in column 1 follows the user with the `ID_CommunityIdentityConnectedTo` given in column 2) is indicated by a “1” in column the `“ID_CommunityConnectionType”`, a ignoring relationship by a “2” in that column (i.e., the user with the `ID_CommunityIdentity` given in column 1 ignores the user with the `ID_CommunityIdentityConnectedTo` given in column 2).

There are different entities in the data set: 
* **Users** - identified by *ID_CommunityIdentity* (or *UserCommunityName*)
* **Postings** - identified by *ID_Posting*
* **Articles** - identified by *ID_Article*

In [ ]:
# define path to datasets:
file_postings_1 = 'data/Postings_01052019_15052019.csv'
file_postings_2 = 'data/Postings_16052019_31052019.csv'
file_votes_1 = 'data/Votes_01052019_15052019.csv'
file_votes_2 = 'data/Votes_16052019_31052019.csv'
file_following_ignoring = 'data/Following_Ignoring_Relationships_01052019_31052019.csv'

output_dir = 'output/'

In [ ]:
df_postings_1 = pd.read_csv(file_postings_1, sep=';')
display(df_postings_1.head(2))
df_postings_1.info()

In [ ]:
df_postings_2 = pd.read_csv(file_postings_2, sep=';')
display(df_postings_2.head(2))
df_postings_2.info()

## Pre-processing Concat Dataset

In [ ]:
# concat postings1 and postings2 in order! 
df_postings = pd.concat([df_postings_1, df_postings_2], ignore_index=True)

df_postings[['PostingCreatedAt', 'ArticlePublishingDate', 'UserCreatedAt']] = df_postings[['PostingCreatedAt', 'ArticlePublishingDate', 'UserCreatedAt']].astype('datetime64')

df_postings.info()
df_postings.head(2)

In [ ]:
# TODO is it necessary to subset the data because it is too large?

#df_postings['PostingCreatedAt'].dt.date.head()
# df_postings = df_postings[pd.to_datetime(df_postings['PostingCreatedAt'].dt.date) == '2019-05-01']
df_postings.shape

## Relation 1: User_A commented/posted to post of User_B

### Extracting Edge relation

Instead of iterating over the dataframe rows to create a edge list, I use a join, which is way faster on the whole dataset. <br>
But, by default `merge` changes the sort order, so you have to sort by the original dataframe index!<br>
The following code creates a dataframe for posting source user and target posting user. It has the same shape as the original dataframe `df_postings`.
So, target posting user may be `NaN`, due to left join and preserving all observations in the posting dataframe.
One could filter the Target_User column by not `NaN` to get only relations between users.

In [ ]:
postings_user_commented_relation_df = df_postings.reset_index()[['index', 'ID_Posting', 'ID_CommunityIdentity', 'PostingCreatedAt', 'ID_Posting_Parent']].merge(
    df_postings[['ID_Posting', 'ID_CommunityIdentity']], 
    left_on='ID_Posting_Parent', right_on='ID_Posting',
    suffixes=('', '_parent'), how='left', sort=False
)[['index', 'ID_Posting', 'PostingCreatedAt', 'ID_CommunityIdentity', 'ID_CommunityIdentity_parent']].sort_values(by='index')

postings_user_commented_relation_df.set_index('index', inplace=True) # index may be useful for selection?
# postings_user_commented_relation_df.set_index(['ID_Posting', 'PostingCreatedAt'], inplace=True) # index may be useful for selection?
postings_user_commented_relation_df.rename(columns={'ID_CommunityIdentity': 'Source_User', 'ID_CommunityIdentity_parent': 'Target_User'}, inplace=True)

# postings_user_commented_relation_df.drop('index', axis='columns', inplace=True) # index is ascending as original df_postings, index column not needed anymore

# SAME number of rows and SAME order as original df_postings:
print(df_postings.shape)
print(postings_user_commented_relation_df.shape)
assert df_postings.shape[0] == postings_user_commented_relation_df.shape[0]

# add column PostingCreatedAtDay to have easy access to Day of month
postings_user_commented_relation_df['PostingCreatedAtDay'] = postings_user_commented_relation_df['PostingCreatedAt'].dt.day

display(postings_user_commented_relation_df.head(5))
display(postings_user_commented_relation_df.info())

In [ ]:
def get_mask_created_at(series, day_start, day_end):
    """
    create a boolean mask of series where day is in day_start incl. and day_end_incl.
    """
    return series.isin(range(day_start, day_end+1))

In [ ]:
mask_posting_created_at = get_mask_created_at(postings_user_commented_relation_df['PostingCreatedAtDay'], 
                                                    day_start=2, day_end=4)

display(postings_user_commented_relation_df.loc[mask_posting_created_at].sample(10))
list(postings_user_commented_relation_df.loc[mask_posting_created_at]['PostingCreatedAtDay'].unique())

Be careful when using the df_posting dataframe without `NAN` values below. Some operations require the original `df_postings` order (NLP text extraction)!

In [ ]:
postings_user_commented_relation_nonan_df = postings_user_commented_relation_df.dropna()
# check number of rows after dropping NAN
print(postings_user_commented_relation_df.shape)
print(postings_user_commented_relation_nonan_df.shape)

### Compute graph topology features

The following function can compute graph features based on posting relation data frame and start and end day:

In [ ]:
def computation_input_df_combined_postings(postings_relation_df, day_start, day_end):
    """
    day_start incl. and day_end incl.
    """
    print('computation_input_df_combined_postings: Days {}-{}'.format(str(day_start),str(day_end)))
    assert (day_start <= day_end) & (day_start > 0) & (day_end <= 31)

    postings_relation_df = postings_relation_df.copy()

    # get postings relations where day is between day_start and day_end
    mask_posting_created_at = get_mask_created_at(postings_relation_df['PostingCreatedAtDay'], 
                                                  day_start=day_start, day_end=day_end)
    postings_relation_df_day_subset = postings_relation_df.loc[mask_posting_created_at]
    # assert all postings are created between day_start and day_end:
    assert postings_relation_df_day_subset['PostingCreatedAtDay'].isin(range(day_start, day_end+1)).all()

    # display(postings_relation_df_day_subset.sample(4))

    # drop NaN in Target_User, because these rows have no relation:
    postings_relation_df_day_subset_no_nan = postings_relation_df_day_subset.dropna().copy()
    postings_relation_df_day_subset_no_nan['Target_User'] = postings_relation_df_day_subset_no_nan['Target_User'].astype('int64')

    #display(postings_relation_df_day_subset_no_nan.sample(4))
    #display(postings_relation_df_day_subset_no_nan.info())

    # obtain interaction count as edge weight
    postings_relation_df_day_subset_no_nan_count = postings_relation_df_day_subset_no_nan[['ID_Posting', 'Source_User', 'Target_User']].groupby(
                                                        ['Source_User', 'Target_User'])['ID_Posting'].count().reset_index(name="weight")

    #display(postings_relation_df_day_subset_no_nan_count.sample(4))
    #display(postings_relation_df_day_subset_no_nan_count.info())

    #Creation of directed graph for day range
    G_postings_commented_days_sub_directed = nx.from_pandas_edgelist(postings_relation_df_day_subset_no_nan_count,
                                                                     source='Source_User',
                                                                     target='Target_User',
                                                                     edge_attr='weight',
                                                                     create_using=nx.DiGraph())
    print('Created a', nx.info(G_postings_commented_days_sub_directed))
    # assert that the graph has weight as edge attribute with a weight > 0 (should be weight column in df)
    assert G_postings_commented_days_sub_directed.get_edge_data(*tuple(postings_relation_df_day_subset_no_nan_count.loc[0, ['Source_User', 'Target_User']]))['weight'] > 0

    G_postings_commented_days_sub_undirected = nx.from_pandas_edgelist(postings_relation_df_day_subset_no_nan_count,
                                                                       source='Source_User',
                                                                       target='Target_User',
                                                                       create_using=nx.Graph())
    print('Created a', nx.info(G_postings_commented_days_sub_undirected))

    # node list, ebunch is pairs of node and should be of length (len(nodes)*(len(nodes)-1)):
    nodes = list(G_postings_commented_days_sub_directed.nodes)
    ebunch = [(i,j) for i,j in product(nodes, nodes) if i!=j] # all node pairs
    assert len(ebunch) == (len(nodes)*(len(nodes)-1))

    print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Starting computation')

    # Computation of Jaccard coefficient
    jac = nx.jaccard_coefficient(G_postings_commented_days_sub_undirected, ebunch)
    jac_df = pd.DataFrame(list(jac), columns = ['Source_User', 'Target_User', 'jaccard_coef'])
    assert jac_df.shape[0] == (len(nodes)*(len(nodes)-1))
    print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Jaccard: ok!')

    # Computation of Adamic and Adar index
    ad_ad = nx.adamic_adar_index(G_postings_commented_days_sub_undirected, ebunch)
    ad_ad_df = pd.DataFrame(list(ad_ad), columns = ['Source_User', 'Target_User', 'adamic_adar_index'])
    assert ad_ad_df.shape[0] == (len(nodes)*(len(nodes)-1))
    print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Adamic and Adar: ok!')
    result_df = jac_df.merge(ad_ad_df)

    # Computation of Preferential Attachment
    pref_att = nx.preferential_attachment(G_postings_commented_days_sub_undirected, ebunch)
    pref_att_df = pd.DataFrame(list(pref_att), columns = ['Source_User', 'Target_User', 'preferential_attachment_index'])
    assert pref_att_df.shape[0] == (len(nodes)*(len(nodes)-1))
    print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Preferential Attachment: ok!')
    result_df = result_df.merge(pref_att_df)

    # Clustering coefficient score (on directed graph)
    cluster_dict = nx.clustering(G_postings_commented_days_sub_directed)
    cluster_dict_df = pd.DataFrame.from_dict(cluster_dict, orient='index', columns=['clustering_coefficient_score']).reset_index().rename(columns={'index':'User'})
    assert cluster_dict_df.shape[0] == len(nodes)
    #display(cluster_dict_df.head())
    # Add a column for clustering_coefficient_score_Source_User using the Clustering coefficient for the Source_User node
    result_df = result_df.merge(cluster_dict_df.rename(columns={'User': 'Source_User', 
                                                               'clustering_coefficient_score': 'clustering_coefficient_score_Source_User'}),
                                on='Source_User', how='left')
    # Add a column for clustering_coefficient_score_Target_User using the Clustering coefficient for the Target_User node
    result_df = result_df.merge(cluster_dict_df.rename(columns={'User': 'Target_User', 
                                                               'clustering_coefficient_score': 'clustering_coefficient_score_Target_User'}),
                                on='Target_User', how='left')
    print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Clustering coefficient: ok!')
    
    # Pagerank (on directed graph)
    pagerank_dict = nx.pagerank(G_postings_commented_days_sub_directed)
    pagerank_dict_df = pd.DataFrame.from_dict(pagerank_dict, orient='index', columns=['pagerank']).reset_index().rename(columns={'index':'User'})
    assert pagerank_dict_df.shape[0] == len(nodes)
     # Add a column for pagerank_Source_User using the Pagerank for the Source_User node
    result_df = result_df.merge(pagerank_dict_df.rename(columns={'User': 'Source_User', 
                                                               'pagerank': 'pagerank_Source_User'}),
                                on='Source_User', how='left')
    # Add a column for pagerank_Target_User using the Pagerank for the Target_User node
    result_df = result_df.merge(pagerank_dict_df.rename(columns={'User': 'Target_User', 
                                                               'pagerank': 'pagerank_Target_User'}),
                                on='Target_User', how='left')
    print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Pagerank: ok!')
    print('Resulting result_df has shape', result_df.shape)
    assert result_df.shape[0] == (len(nodes)*(len(nodes)-1))
    display(result_df.head(5))
    #display(result_df[result_df['clustering_coefficient_score_Target_User'] > 0])

    # Write resulting graph topology features for Source_User and Target_User combinations to a file
    pickle_out_file_path_features = 'output/G_features_postings_days_{}-{}.pkl'.format(str(day_start), str(day_end))
    with open(pickle_out_file_path_features, 'wb') as f:
        pickle.dump(result_df, f)
    print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Successfully written result_df to file "{pickle_out_file_path_features}".')


#### Compute graph features

Calculate for days and save to file. Not needed anymore - file is computed and in Google drive.

In [ ]:
postings_graph_features = computation_input_df_combined_postings(postings_user_commented_relation_df, day_start=1, day_end=3)

## Play with some NLP extractions of text and one hot encoding

In [ ]:
# download spacy german language model - used for tokenization, lemmatization of article text (vector data)
!python -m spacy download de_core_news_sm

### One-Hot encoding (NLP)

One Hot encoding of `'ArticleChannel'`, `'ArticleRessortName'` for ML pipeline:

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# encode article channel and article ressort name with one hot encoding

# TODO use it like that or use the TEST set for one hot encoding. 
# Assuming all article channels and article resort names are known a priori this is ok imho for derStandard.
# Just use a subset of df_posting e.g. certain days, if the number of features is too high in ML pipeline...


# TODO be aware of the order when adding it

# ArticleChannel	ArticleRessortName
one_hot_encoder_article_channel = OneHotEncoder(handle_unknown='ignore').fit(df_postings[['ArticleChannel', 'ArticleRessortName']])

print(one_hot_encoder_article_channel.categories_)

transformed_channel_resort = one_hot_encoder_article_channel.transform(df_postings[['ArticleChannel', 'ArticleRessortName']])
print(transformed_channel_resort.shape)
display(transformed_channel_resort.toarray())

### TF-IDF feature extraction (NLP)

TF-IDF feature extraction of combined `PostingHeadline`, `PostingComment` and `ArticleTitle` next:

In [ ]:
file_name_spacy = 'output/combinedPostingText.spacy'

The following step for creating `'output/combinedPostingText.spacy'` is usually not necessary. It takes around 64 minutes and you can just use my file: <br>
Run the next chunk, if you want to wait long for computing the `"combinedPostingText.spacy"` file.<br>
`doc_bin` holds document information in order of `df_postings`. Existing `combinedPostingText.spacy` stores nlp information for all `df_postings` rows, not just a subset of days. Take care of the order of `df_postings`!

In [ ]:
import spacy
from spacy.tokens import DocBin
nlp = spacy.load("de_core_news_sm")

df_postings['CombinedArticlePostingText'] = df_postings['PostingHeadline'].fillna('') + ' ' + df_postings['PostingComment'].fillna('') + ' ' + df_postings['ArticleTitle'].fillna('')

doc_bin = DocBin(attrs=["LEMMA", "ENT_IOB", "ENT_TYPE"], store_user_data=True)

for doc in nlp.pipe(df_postings['CombinedArticlePostingText'].str.lower()):
    #print(repr(doc))
    doc_bin.add(doc)

#bytes_data = doc_bin.to_bytes()
#doc_bin = DocBin().from_bytes(bytes_data)

doc_bin.to_disk(file_name_spacy)

# save tokenized spacy Doc objects to file
# the file contains the combined text of posting headline, posting comment and article title in a tokenized form
# and also meta data about a token, like its lemma form, whether a token is a punctuation, or bracket etc.
# processing took around 64 minutes on the whole df_postings for me.

The next takes around 5 minutes for me. You could also just use the pickle file `tokenized_lemmatized_texts.pkl`.

In [ ]:
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import spacy
from spacy.tokens import DocBin
nlp = spacy.load("de_core_news_sm")

doc_bin = DocBin().from_disk(file_name_spacy)

docs = list(doc_bin.get_docs(nlp.vocab))
#print(docs)
print(len(docs))
tokenized_lemmatized_texts = [[token.lemma_ for token in doc 
                               if not token.is_stop and not token.is_punct and not token.is_space 
                               and not token.like_url and not token.like_email and not token.is_quote 
                               and not token.is_currency and not token.is_bracket and not token.is_quote
                               and not token.is_left_punct and not token.is_right_punct and not token.is_bracket]
                               for doc in docs]
print(len(tokenized_lemmatized_texts))
print(tokenized_lemmatized_texts[0])
print(tokenized_lemmatized_texts[100])

# tokenized_lemmatized_texts contains nlp information for all documents in df_postings in order!

# save as pickle file:
with open('output/tokenized_lemmatized_texts.pkl', 'wb') as f:
    pickle.dump(tokenized_lemmatized_texts, f)

**Continue here**: Load the pickled file `tokenized_lemmatized_texts.pkl`:

In [ ]:
with open('output/tokenized_lemmatized_texts.pkl', 'rb') as f:
    tokenized_lemmatized_texts = pickle.load(f)

In [ ]:
# tokenized_lemmatized_texts contains nlp information for all documents in df_postings in order!
# is a list of list of tokens, a list of tokens for the combined texts for a posting-article-combination for a certain posting entry in posting_df.
print(tokenized_lemmatized_texts[0:2])
print(len(tokenized_lemmatized_texts))
print(len(df_postings))

assert len(tokenized_lemmatized_texts) == len(df_postings)

I will show an example of utilizing the NLP tf-idf features by adding them to the matrix of graph features.<br>
The example is illustrated with **days 1–3 combined**. We may use these graph feature for Training and Testing!

In [ ]:
# training set - graph based features used are from days 1--3 combined.
# training set - vetor-based NLP features used are from days 1--3:

# filter tokenized_lemmatized_texts by days of PostingCreatedAt:
mask_posting_created_at_days_1to3 = get_mask_created_at(postings_user_commented_relation_df['PostingCreatedAtDay'],
                                                        day_start=1, day_end=3)
# filter target user is not nan
mask_target_user_non_nan = ~postings_user_commented_relation_df['Target_User'].isna()

mask_posting_days_1to3_target_user_non_nan = (mask_posting_created_at_days_1to3 & mask_target_user_non_nan)

tokenized_lemmatized_texts_at_days_1to3 = list(compress(tokenized_lemmatized_texts, list(mask_posting_days_1to3_target_user_non_nan)))
assert (len(tokenized_lemmatized_texts_at_days_1to3)) == mask_posting_days_1to3_target_user_non_nan.sum()
display(tokenized_lemmatized_texts_at_days_1to3[0:1])
print('There are', len(tokenized_lemmatized_texts_at_days_1to3),'tokenized-lemmatized texts for these days where Target_User is not NaN')

assert postings_user_commented_relation_df.loc[mask_posting_days_1to3_target_user_non_nan].isna().sum(axis=0).sum() == 0
assert postings_user_commented_relation_df.loc[mask_posting_days_1to3_target_user_non_nan]['PostingCreatedAtDay'].isin(range(1, 4+1)).all()
assert ~postings_user_commented_relation_df.loc[mask_posting_days_1to3_target_user_non_nan]['PostingCreatedAtDay'].isin(range(5, 31+1)).all()
assert len(tokenized_lemmatized_texts_at_days_1to3) == postings_user_commented_relation_df.loc[mask_posting_days_1to3_target_user_non_nan].shape[0]

In [ ]:
max_tf_idf_features = 500

In [ ]:
# TODO train test split!
# only fit on train data and not on test data!

# fit tokenized-lemmatized text for days subset on train data:

tf_idf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), lowercase=False, tokenizer=lambda x: x, max_features=max_tf_idf_features)
vectorizer = tf_idf_vectorizer.fit(tokenized_lemmatized_texts_at_days_1to3) # training data
print(list(tf_idf_vectorizer.get_feature_names_out()))

In [ ]:
# Obtain tf-idf vectorized text features for training set:

text_tf_idf_features_train = tf_idf_vectorizer.transform(tokenized_lemmatized_texts_at_days_1to3).toarray() # training or test data (here training)
print('TF-IDF vectorized text features have shape:', text_tf_idf_features_train.shape)

assert len(tokenized_lemmatized_texts_at_days_1to3) == text_tf_idf_features_train.shape[0]
assert max_tf_idf_features == text_tf_idf_features_train.shape[1]

In [ ]:
pd.DataFrame(text_tf_idf_features_train).add_suffix('_TFIDF').head()

In [ ]:
# append tf-idf features to the right of postings_user_commented_relation_df content
# BE CAREFUL OF THE ORDER!

display(postings_user_commented_relation_df.loc[mask_posting_days_1to3_target_user_non_nan].reset_index(drop=True).head())
print(postings_user_commented_relation_df.loc[mask_posting_days_1to3_target_user_non_nan].reset_index(drop=True).shape)

display(pd.DataFrame(text_tf_idf_features_train).add_suffix('_TFIDF').head())
print(pd.DataFrame(text_tf_idf_features_train).shape)

# append features
postings_user_commented_relation_df_with_tfidf_features = pd.concat([
    postings_user_commented_relation_df.loc[mask_posting_days_1to3_target_user_non_nan].reset_index(drop=True), 
    pd.DataFrame(text_tf_idf_features_train).add_suffix('_TFIDF')], axis=1, sort=False)

postings_user_commented_relation_df_with_tfidf_features['Target_User'] = postings_user_commented_relation_df_with_tfidf_features['Target_User'].astype('int64')

display(postings_user_commented_relation_df_with_tfidf_features)
print(postings_user_commented_relation_df_with_tfidf_features.shape)

assert postings_user_commented_relation_df_with_tfidf_features.shape[0] == text_tf_idf_features_train.shape[0]
assert postings_user_commented_relation_df_with_tfidf_features.shape[1] == (text_tf_idf_features_train.shape[1] + postings_user_commented_relation_df.loc[mask_posting_days_1to3_target_user_non_nan].shape[1])

In [ ]:
# make sure to copy the pickle file from our shared Google Drive folder
with open('output/G_features_postings_days_1-3.pkl', 'rb') as f:
    G_features_postings_days_1to3 = pickle.load(f)
# dataframe of graph features for Source_User and Target_User
G_features_postings_days_1to3.head()

In [ ]:
postings_tfidf_and_graph_features_only_positive = pd.merge(postings_user_commented_relation_df_with_tfidf_features, G_features_postings_days_1to3,
                                                           on=['Source_User', 'Target_User'], how='left')

In [ ]:
print(postings_tfidf_and_graph_features_only_positive.shape)
display(postings_tfidf_and_graph_features_only_positive.head())
print('Number of NaN in dataframe:')
display(postings_tfidf_and_graph_features_only_positive.isna().sum()[postings_tfidf_and_graph_features_only_positive.isna().sum() > 0])

We can just drop rows with NaN. 934 rows with NaN values is not much compared to 46529 rows in total. (2%).

In [ ]:
display(postings_tfidf_and_graph_features_only_positive.head())
print(postings_tfidf_and_graph_features_only_positive.shape)

In [ ]:
label_positive = pd.DataFrame(np.ones((postings_tfidf_and_graph_features_only_positive.shape[0], 1)), columns=['label']) # only 1s for positive samples

postings_tfidf_and_graph_features_only_positive_with_label = pd.concat([postings_tfidf_and_graph_features_only_positive, label_positive], axis=1)
assert postings_tfidf_and_graph_features_only_positive_with_label.shape == (postings_tfidf_and_graph_features_only_positive.shape[0], postings_tfidf_and_graph_features_only_positive.shape[1]+1)
postings_tfidf_and_graph_features_only_positive_with_label.head()

In [ ]:
# drop NaN - some metrics are strangely NaN
postings_tfidf_and_graph_features_only_positive_with_label.dropna(inplace=True)

# drop posting id column
postings_tfidf_and_graph_features_only_positive_with_label.drop('ID_Posting', axis=1, inplace=True)

In [ ]:
postings_tfidf_and_graph_features_only_positive_with_label.head()

In [ ]:
# save result to file:
with open('output/postings_tfidf_and_graph_features_only_positive_days_1to3.pkl', 'wb') as f:
    pickle.dump(postings_tfidf_and_graph_features_only_positive_with_label, f)

In [ ]:
# load result from file
with open('output/postings_tfidf_and_graph_features_only_positive_days_1to3.pkl', 'rb') as f:
    postings_tfidf_and_graph_features_only_positive_with_label = pickle.load(f)

### negative sampling train

In [ ]:
cols_post_creation_date = ['PostingCreatedAt', 'PostingCreatedAtDay']
cols_source_target_user = ['Source_User', 'Target_User']

In [ ]:
postings_tfidf_and_graph_features_only_positive_with_label['Target_User'].drop_duplicates().sample(frac=2, replace=True)

In [ ]:
# samples for Source_User and Target_User

n = postings_tfidf_and_graph_features_only_positive_with_label.shape[0] * 3
sampled_source_target_user = pd.concat([postings_tfidf_and_graph_features_only_positive_with_label['Source_User'].drop_duplicates().sample(n=n, replace=True, random_state=42).reset_index(drop=True),
                                       postings_tfidf_and_graph_features_only_positive_with_label['Target_User'].drop_duplicates().sample(n=n, replace=True, random_state=42).reset_index(drop=True)], 
                                       axis=1)
print(sampled_source_target_user.shape)


mask_sample_pair_not_in_positives = ~sampled_source_target_user.isin(postings_tfidf_and_graph_features_only_positive_with_label[cols_source_target_user]).all(axis=1)
#display(mask_sample_pair_not_in_positives)

# drop rows where source=target user
mask_source_user_equals_not_target_user = ~((sampled_source_target_user['Source_User'] == sampled_source_target_user['Target_User']))
sampled_source_target_user = sampled_source_target_user.loc[mask_source_user_equals_not_target_user]

sampled_source_target_user = sampled_source_target_user.loc[mask_sample_pair_not_in_positives].reset_index(drop=True)
display(sampled_source_target_user)
print(sampled_source_target_user.shape)
assert sampled_source_target_user.shape[0] > 0 # samples of Source_User and Target_User not in postive dataframe

In [ ]:
# samples for day
n = sampled_source_target_user.shape[0]
sampled_posting_creation_date = postings_tfidf_and_graph_features_only_positive_with_label[cols_post_creation_date].drop_duplicates().sample(n=n, replace=True, random_state=42).reset_index(drop=True)
display(sampled_posting_creation_date)
assert (sampled_posting_creation_date.shape[0] == n) & (sampled_posting_creation_date.shape[1] == 2)

In [ ]:
# concat day and user data:

print(sampled_posting_creation_date.shape)
print(sampled_source_target_user.shape)

display(sampled_posting_creation_date.head())
display(sampled_source_target_user.head())

label_negative = pd.DataFrame(np.zeros((sampled_posting_creation_date.shape[0], 1)), columns=['label']) # only 0s for negative samples

postings_only_negative_with_label = pd.concat([sampled_posting_creation_date, sampled_source_target_user, label_negative], axis=1)

print(postings_only_negative_with_label.shape)
assert (postings_only_negative_with_label.shape[0] == sampled_posting_creation_date.shape[0])
assert (postings_only_negative_with_label.shape[1] == 5)
assert (postings_only_negative_with_label['PostingCreatedAt'].dt.day == postings_only_negative_with_label['PostingCreatedAtDay']).all()
display(postings_only_negative_with_label.sample(10))

In [ ]:
#display(pd.DataFrame(text_tf_idf_features_train).add_suffix('_TFIDF').head())
#print(pd.DataFrame(text_tf_idf_features_train).shape)

n = pd.DataFrame(text_tf_idf_features_train).shape[0]

np.random.seed(42)
sample_tf_idf_features_negative_indices = np.random.choice(np.arange(0, n), size=postings_only_negative_with_label.shape[0], replace=True)
print(sample_tf_idf_features_negative_indices)
assert len(sample_tf_idf_features_negative_indices) == postings_only_negative_with_label.shape[0]

sample_tf_idf_features_negative = pd.DataFrame(text_tf_idf_features_train).add_suffix('_TFIDF').loc[sample_tf_idf_features_negative_indices].reset_index(drop=True)
display(sample_tf_idf_features_negative.head())
print(sample_tf_idf_features_negative.shape)
assert sample_tf_idf_features_negative.shape[0] == postings_only_negative_with_label.shape[0]
assert sample_tf_idf_features_negative.shape[1] == 500
# random sample of tf-idf features for negative examples

In [ ]:
# append features
postings_tfidf_features_only_negative_with_label = pd.concat([
    postings_only_negative_with_label,
    sample_tf_idf_features_negative], axis=1)

postings_tfidf_and_graph_features_only_negative_with_label = pd.merge(postings_tfidf_features_only_negative_with_label, G_features_postings_days_1to3,
                                                           on=['Source_User', 'Target_User'], how='left')

postings_tfidf_and_graph_features_only_negative_with_label.head()

In [ ]:
print(postings_tfidf_and_graph_features_only_negative_with_label.shape)
print(postings_tfidf_and_graph_features_only_positive_with_label.shape)

assert postings_tfidf_and_graph_features_only_positive_with_label.shape[1] == postings_tfidf_and_graph_features_only_negative_with_label.shape[1]

You can sample randomly records from the negative samples dataframe, to balance with the postive number of samples, if necessary.

In [ ]:
# save negative sample result to file:
with open('output/postings_tfidf_and_graph_features_only_negative_days_1to3.pkl', 'wb') as f:
    pickle.dump(postings_tfidf_and_graph_features_only_negative_with_label, f)

In [ ]:
# load result from file
with open('output/postings_tfidf_and_graph_features_only_negative_days_1to3.pkl', 'rb') as f:
    postings_tfidf_and_graph_features_only_negative_with_label = pickle.load(f)